<a href="https://colab.research.google.com/github/eliasab16/MLectric/blob/main/combined_pipeline_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount google drive and load a keras model model

from google.colab import drive
drive.mount('/content/drive')

%cd drive/Othercomputers/My\ MacBook\ Pro/tf-od/panels/

Mounted at /content/drive
/content/drive/Othercomputers/My MacBook Pro/tf-od/panels


In [ ]:
!pip install tensorflowjs

In [ ]:
!apt-get install poppler-utils -y
!pip install pdf2image

In [ ]:
import os
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import pdf2image
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
if os.name=='posix':
    !apt-get install protobuf-compiler -y
    !cd models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
CLASSIFIER_PATH = os.path.join('workspace/classifier')

CUSTOM_MODEL_NAME = 'faster_rcnn'
PRETRAINED_MODEL_NAME = 'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'TRAINED_CLASSIFIERS': os.path.join(CLASSIFIER_PATH, 'trained_models'),
    'WORKSPACE_PATH': os.path.join('workspace'),
    'SCRIPTS_PATH': os.path.join('scripts'),
    'APIMODEL_PATH': os.path.join('models'),
    'ANNOTATION_PATH': os.path.join('workspace','annotated'),
    'IMAGE_PATH': os.path.join('workspace','images'),
    'MODEL_PATH': os.path.join('workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'TFJS_PATH':os.path.join('workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'),
    'TFLITE_PATH':os.path.join('workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('protoc')
}

files = {
    'PIPELINE_CONFIG':os.path.join('workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

### Utils

In [ ]:
labels = {
    0: 'm3x6', 1: 'm3x10', 2: 'm3x16', 3: 'm3x20', 4: 'm3x25', 5: 'm3x32', 6: 'm3x40', 7: 'm3x50', 8: 'm3x63',
    9: 'm1x6', 10: 'm1x10', 11: 'm1x16', 12: 'm1x20', 13: 'm1x32', 14: 'af12', 15: 'af16', 16: 'af26', 17: 'aff',
    18: 'bhaat2x40', 19: 'bhaat4x40', 20: 'b3x40', 21: 'b3x63', 22: 'b3x80', 23: 'b3x100', 24: 'b3x125',
    25: 'b3x160', 26: 'b3x250', 27: 'b3x320'
}

excluded_labels = {
    28: 'sr', 29: 'clock', 30: 'ms', 31: 'm1x+N'
}

hebrew_labels = {
    0: '3x6 מא״ז', 1: '3x10 מא״ז', 2: '3x16 מא״ז', 3: '3x20 מא״ז', 4: '3x25 מא״ז', 5: '3x32 מא״ז', 6: '3x40 מא״', 7: '3x50מא״ז ', 8: '3x63 מא״ז',
    9: '1x6 מא״ז', 10: '1x10 מא״ז', 11: '1x16 מא״ז', 12: '1x20 מא״ז', 13: '1x32 מא״ז', 14: 'AF12 מגען', 15: 'AF16 מגען', 16: 'AF26 מגען', 17: 'AFF מגען',
    18: '2x40 פחת', 19: '4x40 פחת', 20: '3x40 מפסק', 21: '3x63 מפסק', 22: '3x80 מפסק', 23: '3x100 מפסק', 24: '3x125 מפסק',
    25: '3x160 מפסק', 26: '3x250 מפסק', 27: '3x320 מפסק', 28: 'מגע עזר SR', 29: 'שעון', 30: 'MS הגנת מנוע', 31: '1x+N מא״ז'
}

In [ ]:
from pdf2image import convert_from_path
from cv2 import imread

def pdfToImages(pdf_path):
    img_array = []
    images = convert_from_path(pdf_path, 500)

    for img in images:
      img_array.append(np.array(img))

    return img_array

### Load models

#### Detection model

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-34')).expect_partial()

In [ ]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
# mapping excluded class to main label
excluded_classes = {4: 28, 7: 29, 8: 30, 9: 31}

In [ ]:
def detectFromImages(images, detection_threshold=0.9):
  detection_boxes = set() # (detection box array, image index)

  for image_ind, image_np in enumerate(images):
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    label_id_offset = 1
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64) + label_id_offset

    # filter categories
    for detection_index, score in enumerate(detections['detection_scores']):
      if score >= detection_threshold:
        class_id = detections['detection_classes'][detection_index]

        if class_id not in excluded_classes:
          # convert to tuple so that it can be hashed and stored in a set
          box_coord_tuple = tuple(detections['detection_boxes'][detection_index])
          detection_boxes.add((box_coord_tuple, image_ind))
        else:
          # convert excluded class id to main label id
          detected_classes[excluded_classes[class_id]] += 1

  return detection_boxes

In [ ]:
# display image
    # plt.figure(figsize=(14, 10))
    # image_np_with_detections = image_np.copy()

    # viz_utils.visualize_boxes_and_labels_on_image_array(
    #             image_np_with_detections,
    #             detections['detection_boxes'],
    #             detections['detection_classes'],
    #             detections['detection_scores'],
    #             category_index,
    #             use_normalized_coordinates=True,
    #             max_boxes_to_draw=30,
    #             min_score_thresh=.9,
    #             line_thickness=2,
    #             agnostic_mode=False)

    # plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    # plt.show()

#### Classifier model

In [ ]:
classifier = load_model(os.path.join(paths['TRAINED_CLASSIFIERS'], 'alexnet78.h5'))

In [ ]:
tf.keras.models.save_model(classifier, './deployment/classifier')

In [ ]:
!tensorflowjs_converter --input_format=keras_saved_model ./deployment/classifier ./deployment/classifier/classifier_tfjs

## Combine models

In [ ]:
pdf_path = 'workspace/images/raw_data/full_pdf/schematic-1.pdf'

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
%matplotlib inline

detected_classes = {}
for i in range(1, 32):
    detected_classes[i] = 0

images = pdfToImages(pdf_path)

boxes = detectFromImages(images, 0.9)

for box, image_index in boxes:
  box = np.array(box) # box is a tuple of 4 entries (coordinates)
  full_image = images[image_index]

  image_height = full_image.shape[0]
  image_width = full_image.shape[1]

  roi = box*[image_height, image_width, image_height, image_width]
  region = full_image[int(0.99*roi[0]):int(1.01*roi[2]), int(0.99*roi[1]):int(1.01*roi[3])]

  plt.imshow(region)

  # resize to match the input size of the classifier
  region = cv2.resize(region, (256, 256))
  new_region = np.expand_dims(region, axis=0)
  new_region = classifier.predict(new_region, verbose=False)

  prediction = np.argmax(new_region)

  print(f'{hebrew_labels[prediction]}, image: {image_index}')
  plt.show()
  print('_________')

  detected_classes[prediction] += 1


# display results
for class_id in detected_classes:
  if detected_classes[class_id] > 0:
    print(f'{hebrew_labels[class_id]}:')
    print(f'{detected_classes[class_id]}')
    print('________')


## Experimentation code

In [ ]:
height = images[1].shape[0]
width = images[1].shape[1]

In [ ]:
roi = det[3]*[height, width, height, width]
region = images[1][int(0.99*roi[0]):int(1.01*roi[2]), int(0.99*roi[1]):int(1.01*roi[3])]
plt.imshow(region)
plt.show()

In [ ]:
region.shape

In [ ]:
def show_image(ind):
  height = images[ind].shape[0]
  width = images[ind].shape[1]
  roi_np = dets[ind][0]
  image_ind = dets[ind][1]

  roi = roi_np*[height, width, height, width]
  region = images[image_ind][int(0.99*roi[0]):int(1.01*roi[2]), int(0.99*roi[1]):int(1.01*roi[3])]
  plt.imshow(region)
  plt.show()